In [40]:
from notebooks.prep import DATA_DIR
import pandas as pd
import polars as pl
import datetime

In [31]:
parquet_files = [i for i in DATA_DIR.rglob(pattern="*.parquet")]
parquet_files

[PosixPath('/Users/santokalayil/Developer/projects/generative_ai/learn_polars/data/parquet_files/data_4.parquet'),
 PosixPath('/Users/santokalayil/Developer/projects/generative_ai/learn_polars/data/parquet_files/data_5.parquet'),
 PosixPath('/Users/santokalayil/Developer/projects/generative_ai/learn_polars/data/parquet_files/data_7.parquet'),
 PosixPath('/Users/santokalayil/Developer/projects/generative_ai/learn_polars/data/parquet_files/data_6.parquet'),
 PosixPath('/Users/santokalayil/Developer/projects/generative_ai/learn_polars/data/parquet_files/data_3.parquet'),
 PosixPath('/Users/santokalayil/Developer/projects/generative_ai/learn_polars/data/parquet_files/data_2.parquet'),
 PosixPath('/Users/santokalayil/Developer/projects/generative_ai/learn_polars/data/parquet_files/data_9.parquet'),
 PosixPath('/Users/santokalayil/Developer/projects/generative_ai/learn_polars/data/parquet_files/data_0.parquet'),
 PosixPath('/Users/santokalayil/Developer/projects/generative_ai/learn_polars/da

In [32]:
df = pd.concat([pd.read_parquet(i) for i in parquet_files], ignore_index=True)
length, _ = df.shape
print(f"{length:,} rows of data")
df.head()
df.groupby("name_column")["integer_column"].mean()

10,000,000 rows of data


name_column
Alice      49.482394
Bob        49.505852
Emma       49.487737
John       49.474299
Michael    49.492151
Name: integer_column, dtype: float64

In [39]:
df.dtypes

[Datetime(time_unit='ns', time_zone=None),
 String,
 Int64,
 Float64,
 Boolean,
 Categorical(ordering='physical')]

In [34]:
df.memory_usage()

Index                   132
date_column        80000000
name_column        80000000
integer_column     80000000
float_column       80000000
bool_column        10000000
category_column    10000204
dtype: int64

In [35]:
del df

In [48]:
df = pl.scan_parquet(DATA_DIR.rglob(pattern="*.parquet")).collect()

In [49]:
df.head()

date_column,name_column,integer_column,float_column,bool_column,category_column
datetime[ns],str,i64,f64,bool,cat
2024-03-15 18:26:30.420614,"""Alice""",87,0.916001,false,"""D"""
2024-02-04 18:26:30.420635,"""John""",23,0.072863,false,"""D"""
2024-03-02 18:26:30.420637,"""Alice""",18,0.242335,true,"""D"""
2023-08-24 18:26:30.420639,"""Alice""",62,0.966935,true,"""D"""
2024-01-28 18:26:30.420641,"""Bob""",8,0.436317,true,"""A"""


In [51]:
df["date_column"].max(), df["date_column"].min()

(datetime.datetime(2024, 4, 28, 18, 26, 54, 184901),
 datetime.datetime(2023, 4, 29, 18, 26, 10, 481317))

In [57]:
df.filter(df["date_column"]==df["date_column"].max())

date_column,name_column,integer_column,float_column,bool_column,category_column
datetime[ns],str,i64,f64,bool,cat
2024-04-28 18:26:54.184901,"""Michael""",48,0.758684,true,"""D"""


In [62]:
df.filter(df["date_column"].is_between(datetime.datetime(2024, 1, 1), datetime.datetime(2024, 4, 1)))

date_column,name_column,integer_column,float_column,bool_column,category_column
datetime[ns],str,i64,f64,bool,cat
2024-03-15 18:26:30.420614,"""Alice""",87,0.916001,false,"""D"""
2024-02-04 18:26:30.420635,"""John""",23,0.072863,false,"""D"""
2024-03-02 18:26:30.420637,"""Alice""",18,0.242335,true,"""D"""
2024-01-28 18:26:30.420641,"""Bob""",8,0.436317,true,"""A"""
2024-02-28 18:26:30.420647,"""John""",10,0.742562,false,"""C"""
…,…,…,…,…,…
2024-03-06 18:26:49.704520,"""Michael""",71,0.111697,true,"""B"""
2024-03-24 18:26:49.704523,"""Alice""",84,0.67544,true,"""D"""
2024-03-30 18:26:49.704527,"""Alice""",95,0.565232,false,"""D"""


In [63]:
df.filter(pl.col("date_column").is_between(datetime.datetime(2024, 1, 1), datetime.datetime(2024, 4, 1)))

date_column,name_column,integer_column,float_column,bool_column,category_column
datetime[ns],str,i64,f64,bool,cat
2024-03-15 18:26:30.420614,"""Alice""",87,0.916001,false,"""D"""
2024-02-04 18:26:30.420635,"""John""",23,0.072863,false,"""D"""
2024-03-02 18:26:30.420637,"""Alice""",18,0.242335,true,"""D"""
2024-01-28 18:26:30.420641,"""Bob""",8,0.436317,true,"""A"""
2024-02-28 18:26:30.420647,"""John""",10,0.742562,false,"""C"""
…,…,…,…,…,…
2024-03-06 18:26:49.704520,"""Michael""",71,0.111697,true,"""B"""
2024-03-24 18:26:49.704523,"""Alice""",84,0.67544,true,"""D"""
2024-03-30 18:26:49.704527,"""Alice""",95,0.565232,false,"""D"""


In [65]:
pl.col("date_column").is_between(datetime.datetime(2024, 1, 1), datetime.datetime(2024, 4, 1))

<Expr ['col("date_column").is_between(…'] at 0x117360B10>

In [66]:
df["date_column"].is_between(datetime.datetime(2024, 1, 1), datetime.datetime(2024, 4, 1))

date_column
bool
true
true
true
false
true
…
true
true
false


In [67]:
df.group_by("name_column", maintain_order=True).len()

name_column,len
str,u32
"""Alice""",1999958
"""John""",1998735
"""Bob""",2000607
"""Michael""",1999010
"""Emma""",2001690


In [68]:
df.group_by("name_column", maintain_order=False).len()

name_column,len
str,u32
"""Bob""",2000607
"""Michael""",1999010
"""Emma""",2001690
"""John""",1998735
"""Alice""",1999958


In [70]:
df.group_by("name_column", maintain_order=False).agg(pl.col("integer_column").max())

name_column,integer_column
str,i64
"""Michael""",99
"""Alice""",99
"""Emma""",99
"""Bob""",99
"""John""",99


In [74]:
df.group_by("name_column", maintain_order=False).agg(pl.col("integer_column").max().alias("max_age"), pl.col("integer_column").min().alias("min_age"))

name_column,max_age,min_age
str,i64,i64
"""Emma""",99,0
"""Bob""",99,0
"""Alice""",99,0
"""John""",99,0
"""Michael""",99,0


In [77]:
df.select(pl.col("bool_column"), pl.col("name_column"))

bool_column,name_column
bool,str
false,"""Alice"""
false,"""John"""
true,"""Alice"""
true,"""Alice"""
true,"""Bob"""
…,…
false,"""Emma"""
true,"""Emma"""
false,"""Michael"""


In [86]:
df.vstack(df)

date_column,name_column,integer_column,float_column,bool_column,category_column
datetime[ns],str,i64,f64,bool,cat
2024-03-15 18:26:30.420614,"""Alice""",87,0.916001,false,"""D"""
2024-02-04 18:26:30.420635,"""John""",23,0.072863,false,"""D"""
2024-03-02 18:26:30.420637,"""Alice""",18,0.242335,true,"""D"""
2023-08-24 18:26:30.420639,"""Alice""",62,0.966935,true,"""D"""
2024-01-28 18:26:30.420641,"""Bob""",8,0.436317,true,"""A"""
…,…,…,…,…,…
2024-03-27 18:26:49.704529,"""Emma""",99,0.145936,false,"""B"""
2024-03-11 18:26:49.704530,"""Emma""",32,0.769892,true,"""C"""
2023-06-19 18:26:49.704531,"""Michael""",62,0.729478,false,"""C"""


In [90]:
df.rename({c: f"right_{c}" for c in df.columns})

right_date_column,right_name_column,right_integer_column,right_float_column,right_bool_column,right_category_column
datetime[ns],str,i64,f64,bool,cat
2024-03-15 18:26:30.420614,"""Alice""",87,0.916001,false,"""D"""
2024-02-04 18:26:30.420635,"""John""",23,0.072863,false,"""D"""
2024-03-02 18:26:30.420637,"""Alice""",18,0.242335,true,"""D"""
2023-08-24 18:26:30.420639,"""Alice""",62,0.966935,true,"""D"""
2024-01-28 18:26:30.420641,"""Bob""",8,0.436317,true,"""A"""
…,…,…,…,…,…
2024-03-27 18:26:49.704529,"""Emma""",99,0.145936,false,"""B"""
2024-03-11 18:26:49.704530,"""Emma""",32,0.769892,true,"""C"""
2023-06-19 18:26:49.704531,"""Michael""",62,0.729478,false,"""C"""


In [91]:
pl.concat([df, df], how="vertical")

date_column,name_column,integer_column,float_column,bool_column,category_column
datetime[ns],str,i64,f64,bool,cat
2024-03-15 18:26:30.420614,"""Alice""",87,0.916001,false,"""D"""
2024-02-04 18:26:30.420635,"""John""",23,0.072863,false,"""D"""
2024-03-02 18:26:30.420637,"""Alice""",18,0.242335,true,"""D"""
2023-08-24 18:26:30.420639,"""Alice""",62,0.966935,true,"""D"""
2024-01-28 18:26:30.420641,"""Bob""",8,0.436317,true,"""A"""
…,…,…,…,…,…
2024-03-27 18:26:49.704529,"""Emma""",99,0.145936,false,"""B"""
2024-03-11 18:26:49.704530,"""Emma""",32,0.769892,true,"""C"""
2023-06-19 18:26:49.704531,"""Michael""",62,0.729478,false,"""C"""


In [93]:
pl.concat([df, df.rename({c: f"right_{c}" for c in df.columns})], how="horizontal")

date_column,name_column,integer_column,float_column,bool_column,category_column,right_date_column,right_name_column,right_integer_column,right_float_column,right_bool_column,right_category_column
datetime[ns],str,i64,f64,bool,cat,datetime[ns],str,i64,f64,bool,cat
2024-03-15 18:26:30.420614,"""Alice""",87,0.916001,false,"""D""",2024-03-15 18:26:30.420614,"""Alice""",87,0.916001,false,"""D"""
2024-02-04 18:26:30.420635,"""John""",23,0.072863,false,"""D""",2024-02-04 18:26:30.420635,"""John""",23,0.072863,false,"""D"""
2024-03-02 18:26:30.420637,"""Alice""",18,0.242335,true,"""D""",2024-03-02 18:26:30.420637,"""Alice""",18,0.242335,true,"""D"""
2023-08-24 18:26:30.420639,"""Alice""",62,0.966935,true,"""D""",2023-08-24 18:26:30.420639,"""Alice""",62,0.966935,true,"""D"""
2024-01-28 18:26:30.420641,"""Bob""",8,0.436317,true,"""A""",2024-01-28 18:26:30.420641,"""Bob""",8,0.436317,true,"""A"""
…,…,…,…,…,…,…,…,…,…,…,…
2024-03-27 18:26:49.704529,"""Emma""",99,0.145936,false,"""B""",2024-03-27 18:26:49.704529,"""Emma""",99,0.145936,false,"""B"""
2024-03-11 18:26:49.704530,"""Emma""",32,0.769892,true,"""C""",2024-03-11 18:26:49.704530,"""Emma""",32,0.769892,true,"""C"""
2023-06-19 18:26:49.704531,"""Michael""",62,0.729478,false,"""C""",2023-06-19 18:26:49.704531,"""Michael""",62,0.729478,false,"""C"""


In [2]:
import polars as pl

# Example DataFrame
data = {'x': [1, 2, 3, 4, 5], 'y': [5, 4, 3, 2, 1]}
df = pl.DataFrame(data)

# Plot DataFrame using Polars' native plotting capabilities
df.plot(x='x', y='y', kind='scatter')

:Scatter   [x]   (y)